In [5]:
import pandas as pd
import matplotlib as mlp 
import numpy as np 
from io import StringIO

In [15]:
with open('/Users/hieunguyenphi/Documents/PORTFOLIOS/AI PROJECTS/ML Recommender System/MovieTweetings/latest/ratings.dat', 'r') as f:
    content = f.readlines()
    content = [i.strip().split('::') for i in content]
content 

[['1', '0114508', '8', '1381006850'],
 ['2', '0499549', '9', '1376753198'],
 ['2', '1305591', '8', '1376742507'],
 ['2', '1428538', '1', '1371307089'],
 ['3', '0075314', '1', '1595468524'],
 ['3', '0102926', '9', '1590148016'],
 ['3', '0114369', '10', '1597555347'],
 ['3', '0118715', '8', '1596006798'],
 ['3', '0120737', '8', '1599306720'],
 ['3', '0208092', '5', '1586466072'],
 ['3', '0358273', '9', '1579057827'],
 ['3', '0477348', '6', '1597289003'],
 ['3', '10039344', '5', '1578603053'],
 ['3', '1051906', '6', '1589924916'],
 ['3', '1568346', '6', '1597388722'],
 ['3', '2278388', '8', '1597297732'],
 ['3', '6199572', '3', '1589482483'],
 ['3', '6723592', '8', '1599578941'],
 ['3', '6751668', '9', '1578955697'],
 ['3', '7131622', '8', '1579559244'],
 ['3', '7975244', '6', '1583706305'],
 ['3', '7984734', '7', '1579949842'],
 ['3', '8367814', '8', '1586436354'],
 ['3', '8579674', '10', '1579261830'],
 ['3', '8946378', '8', '1584300027'],
 ['4', '0790636', '8', '1391207279'],
 ['4', '1

In [24]:
dat = pd.DataFrame(content)
dat.columns = 'user_id::movie_id::rating::rating_timestamp'.split('::')
dat['rating_timestamp'] = pd.to_datetime(dat.rating_timestamp, unit = 's')
dat['rating'] = dat.rating.astype(int)
dat.head()

,user_id,movie_id,rating,rating_timestamp
0,1,0114508,8,2013-10-05 21:00:50
1,2,0499549,9,2013-08-17 15:26:38
2,2,1305591,8,2013-08-17 12:28:27
3,2,1428538,1,2013-06-15 14:38:09
4,3,0075314,1,2020-07-23 01:42:04


In [25]:
dat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 921398 entries, 0 to 921397
Data columns (total 4 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   user_id           921398 non-null  object        
 1   movie_id          921398 non-null  object        
 2   rating            921398 non-null  int64         
 3   rating_timestamp  921398 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 28.1+ MB


## How many users and movies are mentioned in the dataset?

In [28]:
print(f'Number of distinct users: {len(dat.user_id.drop_duplicates())} users')
print(f'Number of distinct movies: {len(dat.movie_id.drop_duplicates())} movie_id')

Number of distinct users: 71707 users
Number of distinct movies: 38018 movie_id


## How old are the data points (number of data points per year)?

In [30]:
dat.groupby(dat.rating_timestamp.dt.year)['user_id'].agg('count')

rating_timestamp
2013    170807
2014    156354
2015    123644
2016    112455
2017    105987
2018     81870
2019     74183
2020     63115
2021     32983
Name: user_id, dtype: int64

In [31]:
print(f'Number of data points per year: {dat.shape[0]/len(dat.rating_timestamp.dt.year.drop_duplicates())} points')

Number of data points per year: 102377.55555555556 points


## How popular are popular movies (based on the data, when would you say a movie is popular)?

In [34]:
dat.groupby(dat.movie_id)['user_id'].agg('count').sort_values(ascending=False).describe()

count    38018.000000
mean        24.235836
std        113.268936
min          1.000000
25%          1.000000
50%          2.000000
75%          7.000000
max       3104.000000
Name: user_id, dtype: float64

Only 25% top movies are rated over 7 times while average a movies is rated 24 times. The data is heavily skewed to the right with much high values on number of ratings for a movie. It would safely assume that a popular movie is one whose rating numbers are over 24 times.

## How many long-tail movies are there?

In [37]:
dat.groupby(dat.movie_id)['user_id'].agg('count').sort_values(ascending=False).describe(percentiles=[0.999])

count    38018.000000
mean        24.235836
std        113.268936
min          1.000000
50%          2.000000
99.9%     1656.711000
max       3104.000000
Name: user_id, dtype: float64

In [41]:
groups = dat.groupby(dat.movie_id)['user_id'].agg('count')

39

In [42]:
print(f'Number of long-tail movies: {groups[groups >= 1656].shape[0]} movies')

Number of long-tail movies: 39 movies


## How many movies has each user rated?

In [44]:
print(f'Number of movies having had each user rated: {groups[groups >= 1].shape[0]} movies')

Number of movies having had each user rated: 38018 movies


## If you had a recommender system that only recommended the most popular movies, how good would it be?

In [46]:
print(f'If using most popular movies to recommend, the results is always the movie id of {groups[groups == groups.max()].index[0]}')

If using most popular movies to recommend, the results is always the movie id of 1454468
